### Logistic regression work

#### iteration 1

In [ ]:

y = df['stroke']
df.drop('id', axis = 1)
X = df.loc[:, df.columns != "stroke"]
random_state = 42
X_other, X_test, y_other, y_test = train_test_split(X,y,test_size = 0.2,stratify=y,random_state=random_state)
#X_val, X_test, y_val, y_test = train_test_split(X_other,y_other,train_size = 0.5,stratify=y_other,random_state=random_state)
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=random_state)
for train_index, val_index in kf.split(X_other,y_other):
    X_train = X_other.iloc[train_index]
    y_train = y_other.iloc[train_index]
    X_val = X_other.iloc[val_index]
    y_val = y_other.iloc[val_index]

std_ftrs = ['age']
onehot_ftrs = ["work_type", "smoking_status", 'gender', 'ever_married', 'Residence_type']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'), onehot_ftrs),
        ('std', StandardScaler(), std_ftrs)])

clf = Pipeline(steps=[('preprocessor', preprocessor)]) # for now we only preprocess 
                                                       # later on we will add other steps here

X_train_prep = clf.fit_transform(X_train)
X_val_prep = clf.transform(X_val)
X_test_prep = clf.transform(X_test)
print(type(y_train), type(X_train))

alpha = np.logspace(-2,2,21)
print(alpha)
thetas = []
models = []
train_MSE = np.zeros(len(alpha))
val_MSE = np.zeros(len(alpha))
train_ps = np.zeros(len(alpha))
test_ps = np.zeros(len(alpha))
val_ps = np.zeros(len(alpha))
test_acc_scores = np.zeros(len(alpha))
# do the fit
for i in range(len(alpha)):
    log_reg = LogisticRegression(penalty = "l2", C = 1/alpha[i])
    log_reg.fit(X_train_prep, y_train)
    thetas.append(log_reg.coef_)
    models.append(log_reg)
    y_test_pred = log_reg.predict(X_test_prep)
    y_train_pred = log_reg.predict(X_train_prep)
    y_val_pred = log_reg.predict(X_val_prep)
    train_MSE[i] = mean_squared_error(y_train,y_train_pred)
    val_MSE[i] = mean_squared_error(y_val,y_val_pred)
    #print(type(y_train))
    print(sum(np.isnan(y_test_pred)))
    print(pd.DataFrame([y_test, y_test_pred]))
    print(y_test_pred)
    train_ps[i] = precision_score(y_train, y_train_pred)
    test_ps[i] = precision_score(y_test, y_test_pred)
    test_acc_scores[i] = accuracy_score(y_test, y_test_pred)
print(train_MSE)
print(val_MSE)
print(test_ps)
print(test_acc_scores)
print("Best Alpha: ", alpha[np.argmax(test_acc_scores)])
print('Accuracy Score: ', max(test_acc_scores))


#### iteration 2

In [1]:

y = df['stroke']
df.drop('id', axis = 1)
X = df.loc[:, df.columns != "stroke"]
random_state = 42


std_ftrs = ['age']
onehot_ftrs = ["work_type", "smoking_status", 'gender', 'ever_married', 'Residence_type']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'), onehot_ftrs),
        ('std', StandardScaler(), std_ftrs)])
param_grid = {
              'logisticregression__C': np.reciprocal(np.logspace(-2,2,21)),
                'logisticregression__max_iter': [1000]
                } 

nr_states = 3
thetas = []
final_models = []
train_MSE = np.zeros(nr_states)
val_MSE = np.zeros(nr_states)
train_ps = np.zeros(nr_states)
test_ps = np.zeros(nr_states)
val_ps = np.zeros(nr_states)
test_acc_scores = np.zeros(nr_states)
# do the fit

for i in range(nr_states):
    X_other, X_test, y_other, y_test = train_test_split(X,y,test_size = 0.2,stratify=y,random_state=random_state)
    #X_val, X_test, y_val, y_test = train_test_split(X_other,y_other,train_size = 0.5,stratify=y_other,random_state=random_state)
    kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=random_state)
    #oversampling SMOTE

        
        
    log_reg = LogisticRegression(random_state = 42*i, penalty = "l2")
    
    pipe = make_pipeline(preprocessor, log_reg)
    
    grid = GridSearchCV(pipe, param_grid=param_grid,scoring = 'accuracy',
                        cv=kf, return_train_score = True, n_jobs=-1, verbose=True)
    grid.fit(X_other, y_other)
    results = pd.DataFrame(grid.cv_results_)
    
    
    
    print('best model parameters:',grid.best_params_)
    print('validation score:',grid.best_score_) # this is the mean validation score over all iterations
    # save the model
    final_models.append(grid)
    # calculate and save the test score
    y_test_pred = final_models[-1].predict(X_test)
    test_scores[i] = accuracy_score(y_test,y_test_pred)
    test_ps[i] = precision_score(y_test, y_test_pred)
    print('test score:',test_scores[i])
    print('precison score: ', test_ps[i])
    print(pd.DataFrame((y_train, y_train_pred)))
    #check both accuracy and precision 
    #get baseline accuracy and precsion 
    #reference against algorithm
    #SHAP
    #feature importance
    
    

NameError: name 'df' is not defined

### Random Forest Work

In [ ]:
y = df['stroke']
df.drop('id', axis = 1)
X = df.loc[:, df.columns != "stroke"]

std_ftrs = ['age']
onehot_ftrs = ["work_type", "smoking_status", 'gender', 'ever_married', 'Residence_type']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'), onehot_ftrs),
        ('std', StandardScaler(), std_ftrs)])

param_grid = {
              'model__max_depth': [1, 3, 10, 30, 100], # the max_depth should be smaller or equal than the number of features roughly
              'model__max_features': np.linspace(0.5, 1.0, 4) # linearly spaced between 0.5 and 1
              } 

nr_states = 3
final_models = []

#testing scores
test_ps = np.zeros(nr_states)
test_fb = np.zeros(nr_states)

#validation scores
val_scores = np.zeros(nr_states)
val_fb = np.zeros(nr_states)


for i in range(1, nr_states + 1):
    steps = [
        ('preprocess', preprocessor),
        ('over', SMOTE()),
        ('model', RandomForestClassifier(random_state = 42*i))
        ]

    pipeline = Pipeline(steps = steps)
    X_other, X_test, y_other, y_test = train_test_split(X,y,test_size = 0.2,stratify=y,random_state=i*42)
    
    kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=i*42)
    for train_index, val_index in kf.split(X_other,y_other):
        X_train = X_other.iloc[train_index]
        y_train = y_other.iloc[train_index]
        X_val = X_other.iloc[val_index]
        y_val = y_other.iloc[val_index]
    
    grid = GridSearchCV(pipeline, param_grid=param_grid,scoring = make_scorer(precision_score),
                            cv=kf, return_train_score = True, n_jobs=-1, verbose=True)
    grid.fit(X_other, y_other)
    results = pd.DataFrame(grid.cv_results_)


    print('best model parameters:',grid.best_params_)
    print('mean precision score',grid.best_score_) # this is the mean validation score over all iterations
    
    # save the model
    final_models.append(grid)
    
    # calculate and save the test score
    y_test_pred = final_models[-1].predict(X_test)
    y_val_pred = final_models[-1].predict(X_val)
    
    test_ps[i - 1] = precision_score(y_test, y_test_pred)
    test_fb[i - 1] = fbeta_score(y_test, y_test_pred, beta = 1)
    
    val_scores[i - 1] = precision_score(y_val, y_val_pred)
    val_fb[i - 1] = fbeta_score(y_val, y_val_pred, beta = 1)
    print("random_state: ", str(42*i), "precision score: ", precision_score(y_test, y_test_pred))

print()
print('test precison scores: ', test_ps)
print('test f_beta scores: ', test_fb)
print('validation precision scores: ', val_scores)
print('validation f_beta scores: ', val_fb)

In [ ]:

y = df['stroke']
df.drop('id', axis = 1)
X = df.loc[:, df.columns != "stroke"]
random_state = 42

std_ftrs = ['age']
onehot_ftrs = ["work_type", "smoking_status", 'gender', 'ever_married', 'Residence_type']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'), onehot_ftrs),
        ('std', StandardScaler(), std_ftrs)])

#clf = Pipeline(steps=[('preprocessor', preprocessor)]) # for now we only preprocess 
                                                       # later on we will add other steps here

param_grid = {
              'randomforestclassifier__max_depth': [1, 3, 10, 30, 100], # the max_depth should be smaller or equal than the number of features roughly
              'randomforestclassifier__max_features': [0.5,0.75,1.0] # linearly spaced between 0.5 and 1
              } 

nr_states = 3
test_scores = np.zeros(nr_states)
prec_scores = np.zeros(nr_states)
final_models = []

for i in range(nr_states):
    # first split to separate out the test set
    # splitter for other
    X_other, X_test, y_other, y_test = train_test_split(X,y,test_size = 0.2,stratify=y,random_state=random_state)
    kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=random_state)
    for train_index, val_index in kf.split(X_other,y_other):
        X_train = X_other.iloc[train_index]
        y_train = y_other.iloc[train_index]
        X_val = X_other.iloc[val_index]
        y_val = y_other.iloc[val_index]


    # the classifier
    clf = RandomForestClassifier(random_state = 42*i) # initialize the classifier

    # let's put together a pipeline
    # the pipeline will fit_transform the training set (3 folds), and transform the last fold used as validation
    # then it will train the ML algorithm on the training set and evaluate it on the validation set
    # it repeats this step automatically such that each fold will be an evaluation set once
    pipe = make_pipeline(preprocessor,clf)

    # use GridSearchCV
    # GridSearchCV loops through all parameter combinations and collects the results 
    grid = GridSearchCV(pipe, param_grid=param_grid,scoring = 'accuracy',
                        cv=kf, return_train_score = True, n_jobs=-1, verbose=True)
    
    # this line actually fits the model on other
    grid.fit(X_other, y_other)
    # save results into a data frame. feel free to print it and inspect it
    results = pd.DataFrame(grid.cv_results_)
    #print(results)

    print('best model parameters:',grid.best_params_)
    print('validation score:',grid.best_score_) # this is the mean validation score over all iterations
    # save the model
    final_models.append(grid)
    # calculate and save the test score
    y_test_pred = final_models[-1].predict(X_test)
    test_scores[i] = accuracy_score(y_test,y_test_pred)
    prec_scores[i] = precision_score(y_test,y_test_pred)
    print('Accuracy score: ',test_scores[i])
    print('precision score: ', prec_scores[i])
    

### XGBoost work

In [ ]:


y = df['stroke']
df.drop('id', axis = 1)
X = df.loc[:, df.columns != "stroke"]
random_state = 42
X_other, X_test, y_other, y_test = train_test_split(X,y,test_size = 0.2,stratify=y,random_state=random_state)
#X_val, X_test, y_val, y_test = train_test_split(X_other,y_other,train_size = 0.5,stratify=y_other,random_state=random_state)
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=random_state)
for train_index, val_index in kf.split(X_other,y_other):
    X_train = X_other.iloc[train_index]
    y_train = y_other.iloc[train_index]
    X_val = X_other.iloc[val_index]
    y_val = y_other.iloc[val_index]

std_ftrs = ['age']
onehot_ftrs = ["work_type", "smoking_status", 'gender', 'ever_married', 'Residence_type']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'), onehot_ftrs),
        ('std', StandardScaler(), std_ftrs)])

clf = Pipeline(steps=[('preprocessor', preprocessor)]) # for now we only preprocess 
                                                       # later on we will add other steps here
# fit_transform the training set
X_prep = preprocessor.fit_transform(X_train)
# little hacky, but collect feature names
#feature_names = preprocessor.transformers_[0][-1] + \
 #               list(preprocessor.named_transformers_['cat'][1].get_feature_names(cat_ftrs)) + \
 #               preprocessor.transformers_[2][-1]

df_train = pd.DataFrame(data=X_prep)
print(df_train.shape)

# transform the CV
df_val = preprocessor.transform(X_val)
df_val = pd.DataFrame(data=df_val)
print(df_val.shape)

# transform the test
df_test = preprocessor.transform(X_test)
df_test = pd.DataFrame(data=df_test)
print(df_test.shape)

param_grid = {"learning_rate": [0.03],
              "n_estimators": [10000],
              "seed": [0],
              #"reg_alpha": [0e0, 1e-2, 1e-1, 1e0, 1e1, 1e2],
              #"reg_lambda": [0e0, 1e-2, 1e-1, 1e0, 1e1, 1e2],
              "missing": [np.nan], 
              #"max_depth": [1,3,10,30,100],
              "colsample_bytree": [0.9],              
              "subsample": [0.66]}

XGB = xgboost.XGBClassifier()
XGB.set_params(**ParameterGrid(param_grid)[0])
XGB.fit(df_train,y_train,early_stopping_rounds=50,eval_set=[(df_val, y_val)], verbose=False)
y_val_pred = XGB.predict(df_val)
print('the CV RMSE:',np.sqrt(mean_squared_error(y_val,y_val_pred)))
y_test_pred = XGB.predict(df_test)
print('the test RMSE:',np.sqrt(mean_squared_error(y_test,y_test_pred)))
print('Accuracy Score: ', accuracy_score(y_test, y_test_pred))